In [3]:
import pandas as pd 

df = pd.read_csv("news_training_with_translations_20250529.csv")
df.head(5)

,id,media_name,title,content,label,label_encoded,title_en,content_en
0,7cb200a4-3fe7-4606-a1be-84b4c8f3bedf,tvbs,網傳「歐陽娜娜被取消國籍」！陸委會澄清：虛假不實、惡意偽造,網轉影片稱藝人歐陽娜娜因為發布武統言論，遭台灣當局取消國籍終生不得入境，陸委會今天澄清，影片...,偏綠,1,Rumors of Ouyang Nana’s Nationality Revocation...,A video circulating online claims that artist ...
1,be5fccd9-90a6-486f-82fc-4bbce3bb30fc,tvbs,嘆朝野僵局非因溝通問題 賴清德：要政黨願意才有辦法,本屆立法院朝小野大，朝野僵局難解，外界也有不少聲音盼總統賴清德主動與在野溝通。賴清德今（17...,偏綠,1,Sighing Over Legislative Deadlock Not Due to C...,"In the current Legislative Yuan, where the rul..."
2,14514760-ff84-4c9b-8b30-5260a51ce53a,tvbs,大罷免解國會衝突？ 賴清德喊話在野「不必貼民團標籤」,總統賴清德上任周年，朝野衝突愈發嚴重，各地更掀起大罷免潮。賴清德今（17）日接受專訪表示，人...,偏綠,1,Can Mass Recalls Resolve Legislative Conflicts...,As President Lai Ching-te (Lai Qingde) marks h...
3,c3dea17b-d0e5-4371-b11c-5f1133319fe0,tvbs,美暫緩對台關稅90天 賴清德：有決心為國家爭取最好情況,美國對世界推出對等關稅，台灣則在首輪談判名單中，獲得90天寬限期。總統賴清德今（17）日接受...,偏綠,1,U.S. Grants Taiwan 90-Day Tariff Grace Period;...,The United States has introduced reciprocal ta...
4,ee4865d4-caa2-45de-b11f-c2be2a60b132,tvbs,林昶佐派駐芬蘭代表 藍委批賴政府「酬庸」,日前才以歌手身分回歸閃靈樂團的前立委林昶佐，傳出將被指派出任駐芬蘭代表，對此國民黨立委馬文君...,偏藍,0,Freddy Lim Appointed Representative to Finland...,"Former legislator Freddy Lim (林昶佐), who recent..."


In [4]:
# Count NA cells for each column
na_counts = df.isna().sum()
# Print the counts of NA cells
print("Counts of NA cells in each column:")
print(na_counts)


Counts of NA cells in each column:
id                0
media_name        0
title             0
content           0
label             0
label_encoded     0
title_en         46
content_en        0
dtype: int64


In [11]:
# Filter the rows where title_en column is null
df_filtered = df[df['title_en'].isna()]

In [13]:
# Translate Prompt
prompt = """

你是一位專業的新聞翻譯員，擅長處理台灣的政治新聞，能準確保留原文的語氣與用詞，並以清晰、自然的英文呈現。請將下列繁體中文政治新聞標題翻譯成英文，輸出請只要輸出翻譯後的標題，不要包含任何其他文字或說明。請確保翻譯符合以下要求：

請注意以下翻譯原則：
	1.	忠實呈現政治人物的發言與立場，不做美化或誇大。
	2.	保留專有名詞與機構名稱（如：立法院、國民黨、民進黨），可在首次出現時加上英文說明。
	3.	若有台灣特有文化或制度，請簡要加上註解或以英文描述解釋。
	4.	採用正式、新聞風格的英文語氣（如 Reuters 或 BBC 報導風格）。
	5.	保持段落結構清晰，方便後續編輯使用。

以下是舉例：

⸻

【新聞標題】
大罷免解國會衝突？　賴清德喊話在野「不必貼民團標籤」

翻譯後會是：
Can Mass Recalls Resolve Legislative Gridlock? President Lai Urges Opposition Not to Label Civil Groups

⸻
請開始翻譯：
【新聞標題】
"""


In [16]:
import os
import re
from openai import OpenAI
# Set the model to gpt-4.1-mini
model = "gpt-4.1-mini"
api_key = input("Please enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = api_key
# Ensure you have set your OpenAI API key in the environment variable
# Example function to call gpt-4.1-mini (replace with your actual API call)
def translate_with_gpt41mini(text, prompt):
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a professional translator from Traditional Chinese to English, specializing in news articles.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.3,
        max_tokens=1500,
    )

    translation = response.choices[0].message.content.strip()    
    return translation

# Apply translation to missing title_en
for idx, row in df_filtered.iterrows():
    chinese_title = row['title']
    translation_prompt = prompt + chinese_title
    translated_title = translate_with_gpt41mini(chinese_title, translation_prompt)
    df_filtered.at[idx, 'title_en'] = translated_title

# Show the first few rows of the updated DataFrame
print("Translated titles:")
print(df_filtered[['title', 'title_en']].head(5))


Translated titles:
                                       title   
377           快訊／國民黨基隆市黨部涉偽造連署！主委吳國勝「聲押禁見獲准」  \
709                    快訊／張淵翔「請辭獲准」！　涉盜個資助罷免   
802              快訊／罷免提議涉不實連署　國民黨新北市黨部書記長遭聲押   
1070            快訊／網路瘋傳「陳菊病逝」！高醫火速駁斥　監院人士曝真相   
1142  一週大事／索馬利亞禁台護照入境 白沙屯媽祖進香（2025.4.28-5.4）   

                                               title_en  
377   Breaking: Kuomintang Keelung City Branch Accus...  
709   Breaking News: Chang Yuan-hsiang's Resignation...  
802   Breaking News: Recall Proposal Involves False ...  
1070  Breaking News: Rumors of Chen Chu’s Death Spre...  
1142  Weekly Highlights: Somalia Bans Entry with Tai...  


In [17]:
# Put these translated titles back into the original DataFrame
for idx, row in df_filtered.iterrows():
    original_index = row.name  # Get the original index
    df.at[original_index, 'title_en'] = row['title_en']


In [18]:
# Count the number of rows with NA in any column again
# Count NA cells for each column
na_counts = df.isna().sum()
# Print the counts of NA cells
print("Counts of NA cells in each column:")
print(na_counts)


Counts of NA cells in each column:
id               0
media_name       0
title            0
content          0
label            0
label_encoded    0
title_en         0
content_en       0
dtype: int64


In [19]:
# Export the updated DataFrame to a new CSV file
output_file = "news_training_with_translations.csv"
df.to_csv(output_file, index=False)

刪除標題重複的列，只留下第一個出現的標題


In [1]:
import pandas as pd 

df = pd.read_csv("news_training_with_translations.csv")
df.head(5)

,id,media_name,title,content,label,label_encoded,title_en,content_en
0,7cb200a4-3fe7-4606-a1be-84b4c8f3bedf,tvbs,網傳「歐陽娜娜被取消國籍」！陸委會澄清：虛假不實、惡意偽造,網轉影片稱藝人歐陽娜娜因為發布武統言論，遭台灣當局取消國籍終生不得入境，陸委會今天澄清，影片...,偏綠,1,Rumors of Ouyang Nana’s Nationality Revocation...,A video circulating online claims that artist ...
1,be5fccd9-90a6-486f-82fc-4bbce3bb30fc,tvbs,嘆朝野僵局非因溝通問題 賴清德：要政黨願意才有辦法,本屆立法院朝小野大，朝野僵局難解，外界也有不少聲音盼總統賴清德主動與在野溝通。賴清德今（17...,偏綠,1,Sighing Over Legislative Deadlock Not Due to C...,"In the current Legislative Yuan, where the rul..."
2,14514760-ff84-4c9b-8b30-5260a51ce53a,tvbs,大罷免解國會衝突？ 賴清德喊話在野「不必貼民團標籤」,總統賴清德上任周年，朝野衝突愈發嚴重，各地更掀起大罷免潮。賴清德今（17）日接受專訪表示，人...,偏綠,1,Can Mass Recalls Resolve Legislative Conflicts...,As President Lai Ching-te (Lai Qingde) marks h...
3,c3dea17b-d0e5-4371-b11c-5f1133319fe0,tvbs,美暫緩對台關稅90天 賴清德：有決心為國家爭取最好情況,美國對世界推出對等關稅，台灣則在首輪談判名單中，獲得90天寬限期。總統賴清德今（17）日接受...,偏綠,1,U.S. Grants Taiwan 90-Day Tariff Grace Period;...,The United States has introduced reciprocal ta...
4,ee4865d4-caa2-45de-b11f-c2be2a60b132,tvbs,林昶佐派駐芬蘭代表 藍委批賴政府「酬庸」,日前才以歌手身分回歸閃靈樂團的前立委林昶佐，傳出將被指派出任駐芬蘭代表，對此國民黨立委馬文君...,偏藍,0,Freddy Lim Appointed Representative to Finland...,"Former legislator Freddy Lim (林昶佐), who recent..."


In [2]:
# 刪除標題重複的列，只留下第一個出現的標題

df_unique = df.drop_duplicates(subset=['title'], keep='first')
# Count the number of unique titles
unique_titles_count = df_unique['title'].nunique()
print(f"Number of unique titles: {unique_titles_count}")
# Export the DataFrame with unique titles to a new CSV file
output_file_unique = "news_training_with_translations_0531.csv"
df_unique.to_csv(output_file_unique, index=False)
print(f"Data with unique titles saved to {output_file_unique}")


Number of unique titles: 3170
Data with unique titles saved to news_training_with_translations_0531.csv


In [20]:
# 計算各媒體的標記數量
df_label_counts = df_unique.groupby('media_name')['label'].value_counts().reset_index()
df_label_counts
df_label_counts.to_csv("label_counts_by_media.csv", index=False)

In [21]:
# 找出 content 一樣的欄位
df_duplicates = df_unique[df_unique.duplicated(subset=['content'], keep=False)]
df_duplicates['content']

# print the complete content of the duplicates
for index, row in df_duplicates.iterrows():
    print(f"Index: {index}, Content: {row['content']}\n")

# Drop all duplicates based on 'content' column, do not keep any duplicates
df_no_duplicates = df_unique.drop_duplicates(subset=['content'], keep=False)
# Export the DataFrame without duplicates to a new CSV file
output_file_no_duplicates = "news_training_with_translations.csv"
df_no_duplicates.to_csv(output_file_no_duplicates, index=False)

Index: 13, Content: 你打幾分?賴清德執政周年，全民總體檢
訂閱TVBS新聞網YouTube，獨家內容搶先看
加入TVBS新聞LINE，重點新聞一次看

◤住房趨勢一次看◢

👉全台百大旅宿推薦! 再抽住宿券

👉領取飲品買一送一優惠券

👉開啟食尚玩家APP 更多旅遊資訊

Index: 117, Content: 你打幾分?賴清德執政周年，全民總體檢
訂閱TVBS新聞網YouTube，獨家內容搶先看
加入TVBS新聞LINE，重點新聞一次看

◤住房趨勢一次看◢

👉全台百大旅宿推薦! 再抽住宿券

👉領取飲品買一送一優惠券

👉開啟食尚玩家APP 更多旅遊資訊

Index: 539, Content: 你打幾分?賴清德執政周年，全民總體檢
訂閱TVBS新聞網YouTube，獨家內容搶先看
加入TVBS新聞LINE，重點新聞一次看

◤住房趨勢一次看◢

👉全台百大旅宿推薦! 再抽住宿券

👉領取飲品買一送一優惠券

👉開啟食尚玩家APP 更多旅遊資訊

Index: 547, Content: 你打幾分?賴清德執政周年，全民總體檢
訂閱TVBS新聞網YouTube，獨家內容搶先看
加入TVBS新聞LINE，重點新聞一次看

◤住房趨勢一次看◢

👉全台百大旅宿推薦! 再抽住宿券

👉領取飲品買一送一優惠券

👉開啟食尚玩家APP 更多旅遊資訊



In [22]:
df_no_duplicates['title'].nunique(), df_no_duplicates['title_en'].nunique(), df_no_duplicates['content'].nunique()

(3166, 3163, 3166)

In [28]:
# Find duplicates in 'title_en' columns
df_title_en_duplicates = df_no_duplicates[df_no_duplicates.duplicated(subset=['title_en'], keep=False)] 
# Print the titles that have duplicates in 'title_en'
for index, row in df_title_en_duplicates.iterrows():
    print(f"Index: {index}, Media: {row['media_name']}, Title CH: {row['title']}, Title EN: {row['title_en']}\n Content: {row['content']}\n")

Index: 132, Media: tvbs, Title CH: 罷免伍麗華涉「死亡連署」！　領銜人張芳碩深夜遭聲押, Title EN: Recall of Wu Li-hua Involves “Dead Signatures”! Lead Organizer Chang Fang-shuo Detained Overnight
 Content: 屏檢偵辦民進黨山原立委伍麗華罷免案第一階段提議人名冊涉死亡連署，昨約談11人到案說明。罷免案領銜人張芳碩深夜聲押，另2名被告新台幣10萬元交保，其餘8人請回。

屏東地檢署14日約談犯罪嫌疑人3人、證人8人，合計11人到案說明，近午夜時表示，訊後認定張芳碩涉嫌刑法偽造文書、個人資料保護法非法利用個人資料、公職人員選舉罷免法未經同意偽造、假冒提議等罪，犯罪嫌疑重大，且有勾串、滅證之虞及羈押必要性，諭知當庭逮捕並向法院聲請羈押禁見。

Index: 468, Media: tvbs, Title CH: 蔡英文9日訪立陶宛、丹麥　赴哥本哈根民主峰會演說, Title EN: Tsai Ing-wen to Visit Lithuania and Denmark on May 9, Deliver Speech at Copenhagen Democracy Summit
 Content: 前總統蔡英文在卸任後，於去年10月首次出訪歐洲，如今在美國關稅政策之際，蔡英文又將二度赴歐。蔡英文辦公室發言人蔡舒景今（7）日表示，蔡英文將於5月9日出訪立陶宛，以及前往丹麥哥本哈根，出席第八屆「哥本哈根民主高峰會」並發表演說，傳達台灣將致力與民主盟友持續合作、共同確保區域的安全穩定，也將向民主高峰會多年來對台灣的支持表達感謝。

蔡舒景今天表示，蔡英文將於5月9日啟程前往歐洲，出訪立陶宛及丹麥，期待透過此行，能助益於歐洲各界對於台灣更深的認識，也讓台灣與歐洲民主國家間關係更加緊密與強化。

Index: 1413, Media: 三立新聞, Title CH: 罷免伍麗華涉「死亡連署」！領銜人張芳碩深夜遭聲押, Title EN: Recall of Wu Li-hua Involves “Dead Signatures”! Lead Organizer Chang Fang-shuo Detained Overnight
 C

In [29]:
# Delete index 2520
df_no_duplicates = df_no_duplicates.drop(index=2520)
# Export the final DataFrame to a new CSV file
output_file_final = "news_training_with_translations.csv"

In [30]:
# 計算各媒體的標記數量
df_label_counts = df_no_duplicates.groupby('media_name')['label'].value_counts().reset_index()
df_label_counts
df_label_counts.to_csv("label_counts_by_media.csv", index=False)